In [1]:
import tensorflow as tf

import keras as k
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding,InputLayer
 

Using TensorFlow backend.


In [2]:
def process_sentence(sent):
    s = sent.strip().split()
    sp = None
    for i,word in enumerate(s):
        if word == 'EOS':
            sp = i
            break
    sent_tokens = s[1:sp]
    label = s[-1]
    return sent_tokens, label

In [3]:
from nltk import FreqDist

In [4]:
with open('ATIS_Train.txt','r')as fd:
    data_tr = fd.readlines()

In [5]:
with open('ATIS_Test.txt','r')as fd:
    data_te = fd.readlines()

In [6]:
x_tr_tokens = []
y_tr = []
x_te_tokens = []
y_te = []

for sent in data_tr:
    s, l = process_sentence(sent)
    x_tr_tokens.append(s)
    y_tr.append(l)
for sent in data_te:
    s, l = process_sentence(sent)
    x_te_tokens.append(s)
    y_te.append(l)

In [7]:
len(x_te_tokens)

893

In [8]:
cnt = 0
for l in y_tr:
    if '#' in l:
        cnt+=1

In [9]:
y_tr = [j.split('#')[0] for j in y_tr ]

In [10]:
y_te = [j.split('#')[0] for j in y_te ]

In [11]:
all_tokens = sum(x_tr_tokens,[])

In [12]:
fdist = FreqDist(all_tokens)

In [13]:
x_tr_tokens.__len__()

4978

In [14]:
fdist.most_common(4000)[-1]

('scenario', 1)

In [15]:
vocab = set(all_tokens)

In [16]:
word2idx = dict()

In [17]:
lab2ids = {lab:i for i,lab in enumerate(set(y_tr))}

In [18]:
lab2ids

{'atis_airport': 0,
 'atis_ground_service': 1,
 'atis_flight_no': 2,
 'atis_cheapest': 3,
 'atis_airfare': 4,
 'atis_quantity': 5,
 'atis_distance': 6,
 'atis_capacity': 7,
 'atis_restriction': 8,
 'atis_flight_time': 9,
 'atis_abbreviation': 10,
 'atis_city': 11,
 'atis_airline': 12,
 'atis_flight': 13,
 'atis_meal': 14,
 'atis_aircraft': 15,
 'atis_ground_fare': 16}

In [19]:
word2idx['<pad>'] = 0
word2idx['<unk>'] = 1

In [20]:
for i,word in enumerate(vocab):
    word2idx[word]=i+2

In [21]:
mxlen=0
for sent in x_tr_tokens:
    mxlen = max(len(sent),mxlen)

In [22]:
x_tr_ids = [ [word2idx[word] for word in sent] for sent in x_tr_tokens ]

In [23]:
x_te_ids = [ [word2idx.get(word,1) for word in sent] for sent in x_te_tokens ]

In [24]:
y_tr_idx = np.asarray([lab2ids[lab] for lab in y_tr ])
y_te_idx = np.asarray([lab2ids.get(lab,0) for lab in y_te ])

In [25]:
x_tr_ids_pad = []
for sent in x_tr_ids:
    ln = len(sent)
    toadd = mxlen-ln
    new = [0]*toadd
    new = sent+new
    x_tr_ids_pad.append(np.asarray(new,dtype = np.int32))

In [26]:
x_te_ids_pad = []
for sent in x_te_ids:
    sent = sent[:mxlen]
    ln = len(sent)
    toadd = mxlen-ln
    new = [0]*toadd
    new = sent+new
    x_te_ids_pad.append(np.asarray(new,dtype = np.int32))

In [27]:
x_te_ids_pad = np.asarray(x_te_ids_pad)
x_tr_ids_pad = np.asarray(x_tr_ids_pad)

In [29]:
x_te_ids_pad.shape

(893, 46)

In [30]:
x_tr_ids_pad.shape

(4978, 46)

In [31]:
y_tr_cat = to_categorical(y_tr_idx, num_classes=17)
y_te_cat = to_categorical(y_te_idx, num_classes=17)

In [32]:
act = [('relu','relu'),('','relu'),('relu','relu'),('relu','relu')]
print('binary with sigmoid')
model = Sequential()
model.add(InputLayer(input_shape=(mxlen,)))
model.add(Embedding(len(word2idx),100))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=17, activation='sigmoid'))
 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(x_tr_ids_pad, y_tr_cat, 
          epochs=3, batch_size=512, verbose=1,
          validation_split=0.2)
 


scores = model.evaluate(x_te_ids_pad, y_te_cat, verbose=1)
print("Accuracy:", scores[1]) 


binary with sigmoid
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 46, 100)           90000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 42, 64)            32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_2 (Dense)              (None, 17)                1105      
Total params: 156,001
Trainable params: 156,001
Non-trainable params: 0
____________________________

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3982 samples, validate on 996 samples
Epoch 1/3
3982/3982 [==============================] - 1s 277us/step - loss: 0.6537 - accuracy: 0.7395 - val_loss: 0.5700 - val_accuracy: 0.8798
Epoch 2/3
3982/3982 [==============================] - 0s 62us/step - loss: 0.4514 - accuracy: 0.9435 - val_loss: 0.2721 - val_accuracy: 0.9681
Epoch 3/3
893/893 [==============================] - 0s 46us/step
Accuracy: 0.9673275351524353


In [47]:
act = [('relu','relu'),('tanh','relu'),('tanh','tanh'),('relu','tanh')]
print('categorical with softmax')
for cfg in act:
    model = Sequential()
    model.add(InputLayer(input_shape=(mxlen,)))
    model.add(Embedding(len(word2idx),100))
    model.add(Conv1D(64, 5, activation=cfg[0]))
    model.add(MaxPooling1D(5))
    model.add(Flatten())
    model.add(Dense(units=64, activation=cfg[1]))++
    model.add(Dense(units=17, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())

    model.fit(x_tr_ids_pad, y_tr_cat, 
              epochs=30, batch_size=512, verbose=0,
              validation_split=0.2)



    scores = model.evaluate(x_te_ids_pad, y_te_cat, verbose=1)
    y_hat = model.predict(x_te_ids_pad)

    print("Accuracy of config {}:{}".format(cfg, scores[1]))
    get_report(y_te_cat,y_hat,tar_names)

categorical with softmax
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 46, 100)           90000     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 42, 64)            32064     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_10 (Dense)             (None, 17)                1105      
Total params: 156,001
Trainable params: 156,001
Non-trainable params: 0
_______________________

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


893/893 [==============================] - 0s 35us/step
Accuracy of config ('relu', 'relu'):0.9440089464187622
                     precision    recall  f1-score   support

       atis_airport       0.94      0.75      0.83        20
atis_ground_service       0.97      1.00      0.99        36
     atis_flight_no       0.00      0.00      0.00         9
      atis_cheapest       0.94      0.98      0.96        49
       atis_airfare       0.30      1.00      0.46         3
      atis_quantity       1.00      0.60      0.75        10
      atis_distance       0.94      0.81      0.87        21
      atis_capacity       0.20      1.00      0.33         1
   atis_restriction       0.97      1.00      0.99        33
   atis_flight_time       0.00      0.00      0.00         6
  atis_abbreviation       0.93      0.97      0.95        38
          atis_city       0.96      0.99      0.97       645
       atis_airline       0.00      0.00      0.00         6
        atis_flight       0.83    

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1870: UserWarning: labels size, 15, does not match size of target_names, 16
  .format(len(labels), len(target_names))
/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


893/893 [==============================] - 0s 36us/step
Accuracy of config ('tanh', 'relu'):0.8902575373649597
                     precision    recall  f1-score   support

       atis_airport       1.00      0.15      0.26        20
atis_ground_service       0.67      0.97      0.80        36
     atis_flight_no       0.00      0.00      0.00         9
      atis_cheapest       0.73      0.92      0.81        49
       atis_airfare       0.00      0.00      0.00         3
      atis_quantity       0.00      0.00      0.00        10
      atis_distance       1.00      0.05      0.09        21
      atis_capacity       0.17      1.00      0.29         1
   atis_restriction       0.79      1.00      0.88        33
   atis_flight_time       0.00      0.00      0.00         6
  atis_abbreviation       0.89      0.87      0.88        38
          atis_city       0.95      0.99      0.97       645
       atis_airline       0.00      0.00      0.00         6
        atis_flight       0.45    

...

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


893/893 [==============================] - 0s 37us/step
Accuracy of config ('tanh', 'tanh'):0.8947368264198303
                     precision    recall  f1-score   support

       atis_airport       0.00      0.00      0.00        20
atis_ground_service       0.72      1.00      0.84        36
     atis_flight_no       0.00      0.00      0.00         9
      atis_cheapest       0.87      0.80      0.83        49
       atis_airfare       0.17      0.33      0.22         3
      atis_quantity       0.00      0.00      0.00        10
      atis_distance       1.00      0.38      0.55        21
      atis_capacity       0.00      0.00      0.00         1
   atis_restriction       0.70      1.00      0.82        33
   atis_flight_time       0.50      0.33      0.40         6
  atis_abbreviation       0.94      0.89      0.92        38
          atis_city       0.93      0.99      0.96       645
       atis_airline       0.00      0.00      0.00         6
        atis_flight       0.56    

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1870: UserWarning: labels size, 15, does not match size of target_names, 16
  .format(len(labels), len(target_names))
/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


893/893 [==============================] - 0s 42us/step
Accuracy of config ('relu', 'tanh'):0.9316909313201904
                     precision    recall  f1-score   support

       atis_airport       1.00      0.20      0.33        20
atis_ground_service       0.78      1.00      0.88        36
     atis_flight_no       0.00      0.00      0.00         9
      atis_cheapest       0.94      1.00      0.97        49
       atis_airfare       0.30      1.00      0.46         3
      atis_quantity       0.75      0.60      0.67        10
      atis_distance       0.88      0.71      0.79        21
      atis_capacity       0.33      1.00      0.50         1
   atis_restriction       1.00      1.00      1.00        33
   atis_flight_time       0.00      0.00      0.00         6
  atis_abbreviation       0.95      0.97      0.96        38
          atis_city       0.96      0.99      0.97       645
       atis_airline       0.00      0.00      0.00         6
        atis_flight       1.00    

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1870: UserWarning: labels size, 15, does not match size of target_names, 16
  .format(len(labels), len(target_names))
/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
def get_report(y_true,y_hat,target_names):
    y_h = np.argmax(y_hat,axis=1)
    y_t = np.argmax(y_true,axis=1)
    print(classification_report(y_t, y_h, target_names=target_names[:16],labels = list(set(y_t))))

In [35]:
! pip install scipy

In [36]:
np.argmax(y_hat,axis=1).shape

(893,)

In [37]:
idx2lab = {j:i for i,j in lab2ids.items()}

In [38]:
tar_names = list(idx2lab.values())

In [39]:
y_hat.shape

(893, 17)

In [40]:
y_te_cat.shape

(893, 17)

In [49]:
act = [('relu','relu'),('tanh','relu'),('tanh','tanh'),('relu','tanh')]
print('categorical with softmax')
for cfg in act:
    model = Sequential()
    model.add(InputLayer(input_shape=(mxlen,)))
    model.add(Embedding(len(word2idx),100))
    model.add(Conv1D(64, 5, activation=cfg[0]))
    model.add(MaxPooling1D(5))
    model.add(Flatten())
    model.add(Dense(units=64, activation=cfg[1]))
    model.add(Dense(units=17, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    print(model.summary())

    model.fit(x_tr_ids_pad, y_tr_cat, 
              epochs=30, batch_size=512, verbose=0,
              validation_split=0.2)



    scores = model.evaluate(x_te_ids_pad, y_te_cat, verbose=1)
    y_hat = model.predict(x_te_ids_pad)

    print("Accuracy of config {}:{}".format(cfg, scores[1]))
    get_report(y_te_cat,y_hat,tar_names)

categorical with softmax
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 46, 100)           90000     
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 42, 64)            32064     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 8, 64)             0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_20 (Dense)             (None, 17)                1105      
Total params: 156,001
Trainable params: 156,001
Non-trainable params: 0
______________________

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


893/893 [==============================] - 0s 49us/step
Accuracy of config ('relu', 'relu'):0.7222844362258911
                     precision    recall  f1-score   support

       atis_airport       0.00      0.00      0.00        20
atis_ground_service       0.00      0.00      0.00        36
     atis_flight_no       0.00      0.00      0.00         9
      atis_cheapest       0.00      0.00      0.00        49
       atis_airfare       0.00      0.00      0.00         3
      atis_quantity       0.00      0.00      0.00        10
      atis_distance       0.00      0.00      0.00        21
      atis_capacity       0.00      0.00      0.00         1
   atis_restriction       0.00      0.00      0.00        33
   atis_flight_time       0.00      0.00      0.00         6
  atis_abbreviation       0.00      0.00      0.00        38
          atis_city       0.72      1.00      0.84       645
       atis_airline       0.00      0.00      0.00         6
        atis_flight       0.00    

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1870: UserWarning: labels size, 15, does not match size of target_names, 16
  .format(len(labels), len(target_names))
/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 46, 100)           90000     
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 42, 64)            32064     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 8, 64)             0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_22 (Dense)             (None, 17)                1105      
Total params: 156,001
Trainable params: 156,001
Non-trainable params: 0
_______________________________________________

...

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1870: UserWarning: labels size, 15, does not match size of target_names, 16
  .format(len(labels), len(target_names))
/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 46, 100)           90000     
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 42, 64)            32064     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 8, 64)             0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_26 (Dense)             (None, 17)                1105      
Total params: 156,001
Trainable params: 156,001
Non-trainable params: 0
_______________________________________________

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


893/893 [==============================] - 0s 49us/step
Accuracy of config ('relu', 'tanh'):0.7222844362258911
                     precision    recall  f1-score   support

       atis_airport       0.00      0.00      0.00        20
atis_ground_service       0.00      0.00      0.00        36
     atis_flight_no       0.00      0.00      0.00         9
      atis_cheapest       0.00      0.00      0.00        49
       atis_airfare       0.00      0.00      0.00         3
      atis_quantity       0.00      0.00      0.00        10
      atis_distance       0.00      0.00      0.00        21
      atis_capacity       0.00      0.00      0.00         1
   atis_restriction       0.00      0.00      0.00        33
   atis_flight_time       0.00      0.00      0.00         6
  atis_abbreviation       0.00      0.00      0.00        38
          atis_city       0.72      1.00      0.84       645
       atis_airline       0.00      0.00      0.00         6
        atis_flight       0.00    

/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1870: UserWarning: labels size, 15, does not match size of target_names, 16
  .format(len(labels), len(target_names))
/home/stud/btech/cse/2016/rsrivatsa.cs16/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [42]:
y_t = np.argmax(y_te_cat,axis=1)

In [43]:
len(set(y_t))

15